# SMA Strategy Backtesting System
This notebook combines the functionality from multiple files into a single interactive notebook format.
The system implements a Simple Moving Average (SMA) crossover strategy with ATR-based position sizing.

## Imports and Setup
Import required libraries and set up basic configurations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import json
import time
import struct
from typing import Dict, Tuple, List
import seaborn as sns
import matplotlib.lines as mlines
from matplotlib.patches import Circle
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import calendar
import openpyxl
import input  # Import the module instead of individual variables
from scipy.spatial.distance import cdist
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist
from matplotlib.lines import Line2D

# Import configuration
from input import MIN_TRADES, MAX_TRADES, MIN_ELEMENTS_PER_CLUSTER, DEFAULT_NUM_CLUSTERS
from input import TRAIN_TEST_SPLIT, ATR_PERIOD, TRADING_CAPITAL, TICKER

# Define SYMBOL globally
SYMBOL = TICKER.replace('=F', '')

# Setup paths using relative directories
WORKING_DIR = "."
DATA_DIR = os.path.join(WORKING_DIR, "data")

def save_results(data_frame, file_name):
    """Save results DataFrame to a CSV file"""
    data_frame.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")

## Utility Functions from read_ts.py
Only including the essential functions that are used in the strategy

In [ ]:
class mdata:
    """Data structure for holding market data and metadata"""
    tick_size = None
    big_point_value = None
    country = None
    exchange = None
    symbol = None
    description = None
    interval_type = None
    interval_span = None
    time_zone = None
    session = None
    data = None

    def __repr__(self):
        return str((self.symbol, self.interval_type, self.interval_span, self.data.shape))

def read_ts_ohlcv_dat_one_stream(byte_stream) -> mdata:
    """Read a single OHLCV data stream"""
    def read_string(f):
        sz = struct.unpack('i', f.read(4))[0]
        s = f.read(sz).decode('ascii')
        return s
    
    d = mdata()
    try:
        f = byte_stream
        (ones, type_format) = struct.unpack('ii', f.read(8))
        if (ones != 1111111111):
            print("format not supported, must be 1111111111")
            return None
        if (type_format != 3):
            print("type_format not supported, must be 3")
            return None
        d.tick_size = struct.unpack('d', f.read(8))[0]
        d.big_point_value = struct.unpack('d', f.read(8))[0]
        d.country = read_string(f)
        d.exchange = read_string(f)
        d.symbol = read_string(f)
        d.description = read_string(f)
        d.interval_type = read_string(f)
        d.interval_span = struct.unpack('i', f.read(4))[0]
        d.time_zone = read_string(f)
        d.session = read_string(f)

        dt = np.dtype([('date', 'f8'), ('open', 'f8'), ('high', 'f8'), ('low', 'f8'), ('close', 'f8'), ('volume', 'f8')])
        z = f.read()
        data = pd.DataFrame.from_records(np.frombuffer(z, dtype=dt))
    finally:
        pass

    arr = ((data['date']-25569)*24*60*60).round(0).astype(np.int64)*1000000000
    z2 = pd.to_datetime(arr)
    data.insert(0, 'datetime', z2)
    del data['date']
    d.data = data
    return d

def read_ts_ohlcv_dat_one(fname) -> mdata:
    """Read a single OHLCV data file"""
    try:
        f = open(fname, "rb")
        d = read_ts_ohlcv_dat_one_stream(f)
    finally:
        f.close()
    return d

def read_ts_ohlcv_dat(fnames) -> List[mdata]:
    """Read multiple OHLCV data files"""
    r = []
    for name in glob.glob(fnames):
        z = read_ts_ohlcv_dat_one(name)
        r.append(z)
    return r

def find_futures_file(symbol, data_dir):
    """Find a data file for the specified futures symbol"""
    pattern = f"*@{symbol}_*.dat"
    files = glob.glob(os.path.join(data_dir, pattern))
    
    if not files:
        pattern = f"*_@{symbol}_*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
    
    if not files:
        pattern = f"*_{symbol}_*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
    
    if not files:
        pattern = f"*@{symbol}*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
    
    if not files:
        raise FileNotFoundError(f"No data file found for {symbol} in {data_dir}")
        
    return files[0]

def mavg(d, period):
    """Calculate moving average"""
    return np.concatenate([np.zeros((period-1)), np.convolve(d, np.ones((period,))/period, mode='valid')]) 

def analyze_sma_results(file_path='sma_all_results.txt'):
    """Analyze SMA optimization results and return best parameters"""
    try:
        # Load the data from the CSV file
        data = pd.read_csv(file_path)
        
        print(f"Loaded {len(data)} simulation results")
        print(f"Short SMA range: {data['short_SMA'].min()} to {data['short_SMA'].max()}")
        print(f"Long SMA range: {data['long_SMA'].min()} to {data['long_SMA'].max()}")
        print(f"Sharpe ratio range: {data['sharpe_ratio'].min():.4f} to {data['sharpe_ratio'].max():.4f}")
        
        # Apply filtering before finding best parameters
        filtered_data = data[
            (data['short_SMA'] < data['long_SMA']) &  # Ensure short_SMA < long_SMA
            (data['trades'] >= MIN_TRADES) &  # Minimum trades requirement
            (data['trades'] <= MAX_TRADES)    # Maximum trades requirement
        ]

        if len(filtered_data) == 0:
            print(f"No data points meet the criteria after filtering! Adjust min_trades ({MIN_TRADES}) and max_trades ({MAX_TRADES}).")
            return None, None, None, None, None

        print(f"\nAfter filtering:")
        print(f"Remaining data points: {len(filtered_data)}")
        print(f"Short SMA range: {filtered_data['short_SMA'].min()} to {filtered_data['short_SMA'].max()}")
        print(f"Long SMA range: {filtered_data['long_SMA'].min()} to {filtered_data['long_SMA'].max()}")
        print(f"Sharpe ratio range: {filtered_data['sharpe_ratio'].min():.4f} to {filtered_data['sharpe_ratio'].max():.4f}")
        print(f"Trades range: {filtered_data['trades'].min()} to {filtered_data['trades'].max()}")
        
        # Find the best Sharpe ratio from filtered data
        max_sharpe_idx = filtered_data['sharpe_ratio'].idxmax()
        best_row = filtered_data.iloc[max_sharpe_idx]
        
        print(f"\nBest parameters (after filtering):")
        print(f"Short SMA: {int(best_row['short_SMA'])}")
        print(f"Long SMA: {int(best_row['long_SMA'])}")
        print(f"Sharpe Ratio: {best_row['sharpe_ratio']:.6f}")
        print(f"Number of Trades: {int(best_row['trades'])}")
        
        return (filtered_data, 
                int(best_row['short_SMA']), 
                int(best_row['long_SMA']), 
                best_row['sharpe_ratio'], 
                int(best_row['trades']))
    except Exception as e:
        print(f"Error analyzing SMA results: {e}")
        return None, None, None, None, None

## SMA Strategy Implementation
Implementation of the Simple Moving Average strategy with ATR-based position sizing

In [ ]:
class SMAStrategy:
    """
    Vectorized SMA (Simple Moving Average) trading strategy with ATR-based position sizing
    
    Features:
    - Basic SMA crossover entry signals (long when short SMA crosses above long SMA, short when it crosses below)
    - Pure long/short positions based on crossover direction
    - Position size based on ATR volatility
    - Slippage modeling for realistic execution
    - P&L calculated in absolute dollar terms using big point value
    - Fully vectorized implementation for efficiency
    """

    def __init__(self, short_sma, long_sma, big_point_value, slippage=0, capital=6000, atr_period=50):
        """
        Initialize the SMA strategy with specific parameters

        Parameters:
        short_sma (int): Short SMA period in days
        long_sma (int): Long SMA period in days
        big_point_value (int): Contract big point value for calculating dollar P&L
        slippage (float): Slippage in price units added/subtracted from execution price
        capital (float): The capital allocation for position sizing
        atr_period (int): Period for ATR calculation for position sizing
        """
        self.short_sma = short_sma
        self.long_sma = long_sma
        self.big_point_value = big_point_value
        self.slippage = slippage
        self.capital = capital
        self.atr_period = atr_period

    def calculate_atr(self, data, period=None):
        """
        Calculate Average True Range (ATR)
        
        Parameters:
        data: DataFrame with OHLC data
        period: ATR calculation period, defaults to self.atr_period if None
        
        Returns:
        Series: ATR values
        """
        if period is None:
            period = self.atr_period
            
        # Calculate True Range
        high = data['High'].copy()
        prev_close = data['Close'].shift(1)
        true_high = pd.DataFrame({'high': high, 'prev_close': prev_close}).max(axis=1)
        
        low = data['Low'].copy()
        true_low = pd.DataFrame({'low': low, 'prev_close': prev_close}).min(axis=1)
        
        true_range = true_high - true_low
        atr = true_range.rolling(window=period).mean()
        
        return atr

    def apply_strategy(self, data, strategy_name="Strategy"):
        """
        Apply the simple SMA crossover strategy to the price data using vectorized operations
        with ATR-based position sizing
        """
        sim_data = data.copy()
        
        sim_data[f'SMA_Short_{strategy_name}'] = mavg(sim_data['Close'].values, self.short_sma)
        sim_data[f'SMA_Long_{strategy_name}'] = mavg(sim_data['Close'].values, self.long_sma)
        
        sim_data[f'ATR_{strategy_name}'] = self.calculate_atr(sim_data, self.atr_period)
        
        sim_data[f'Position_Size_{strategy_name}'] = np.round(
            self.capital / (sim_data[f'ATR_{strategy_name}'] * self.big_point_value) + 0.5
        )
        
        sim_data[f'Position_Dir_{strategy_name}'] = np.where(
            sim_data[f'SMA_Short_{strategy_name}'] > sim_data[f'SMA_Long_{strategy_name}'], 1, -1
        )
        
        sim_data[f'Position_Dir_{strategy_name}'] = sim_data[f'Position_Dir_{strategy_name}'].fillna(0)
        
        sim_data[f'Position_Change_{strategy_name}'] = sim_data[f'Position_Dir_{strategy_name}'].diff() != 0
        
        market_pnl = sim_data['Close'].diff() * self.big_point_value
        sim_data[f'Market_PnL_{strategy_name}'] = market_pnl
        
        sim_data[f'Daily_PnL_{strategy_name}'] = (
            market_pnl * 
            sim_data[f'Position_Dir_{strategy_name}'].shift(1) * 
            sim_data[f'Position_Size_{strategy_name}'].shift(1)
        )
        
        position_changed = sim_data[f'Position_Change_{strategy_name}']
        sim_data.loc[position_changed, f'Daily_PnL_{strategy_name}'] -= (
            self.slippage * sim_data[f'Position_Size_{strategy_name}'][position_changed]
        )
        
        sim_data[f'Daily_PnL_{strategy_name}'] = sim_data[f'Daily_PnL_{strategy_name}'].fillna(0)
        
        sim_data[f'Cumulative_PnL_{strategy_name}'] = sim_data[f'Daily_PnL_{strategy_name}'].cumsum()
        
        return sim_data 

    def optimize(self, data, sma_range, train_test_split=0.7, results_file=None, warm_up_idx=None):
        """
        Find the optimal SMA parameters and record all simulations using vectorized operations

        Parameters:
        data: DataFrame with market data
        sma_range: Range of SMA periods to test
        train_test_split: Portion of data to use for in-sample testing
        results_file: Path to save simulation results
        warm_up_idx: Index to trim warm-up period (if provided)

        Returns:
        best_sma_params: Tuple with (short_sma, long_sma)
        best_sharpe: Best Sharpe ratio found
        best_trades: Number of trades with best parameters
        all_results: List of tuples with all simulation results
        """
        best_sharpe = -np.inf
        best_sma = (0, 0)
        best_trades = 0
        best_sim_data = None
        all_results = []

        if results_file:
            with open(results_file, 'w') as f:
                f.write("short_SMA,long_SMA,trades,sharpe_ratio\n")

        total_combinations = sum(1 for a, b in [(s, l) for s in sma_range for l in sma_range] if a < b)
        completed = 0
        total_sim_time = 0
        
        print(f"Running {total_combinations} simulations...")
        
        for short_sma in sma_range:
            for long_sma in sma_range:
                if short_sma >= long_sma:
                    continue

                sim_start_time = time.time()
                orig_short_sma = self.short_sma
                orig_long_sma = self.long_sma
                self.short_sma = short_sma
                self.long_sma = long_sma

                sim_data = self.apply_strategy(data.copy(), strategy_name="Sim")

                if warm_up_idx is not None:
                    sim_data_eval = sim_data.iloc[warm_up_idx:].copy()
                    split_index = int(len(sim_data_eval) * train_test_split)
                else:
                    sim_data_eval = sim_data
                    split_index = int(len(sim_data_eval) * train_test_split)

                trade_entries = sim_data_eval['Position_Change_Sim']
                trade_count = trade_entries.sum()

                sim_data_eval.loc[:, 'Daily_Returns'] = sim_data_eval['Daily_PnL_Sim']
                in_sample_returns = sim_data_eval['Daily_Returns'].iloc[:split_index]

                if len(in_sample_returns.dropna()) == 0 or in_sample_returns.std() == 0:
                    sharpe_ratio = 0
                else:
                    sharpe_ratio = in_sample_returns.mean() / in_sample_returns.std() * np.sqrt(252)

                if results_file:
                    with open(results_file, 'a') as f:
                        f.write(f"{short_sma},{long_sma},{trade_count},{sharpe_ratio:.6f}\n")

                result = (short_sma, long_sma, trade_count, sharpe_ratio)
                all_results.append(result)

                if sharpe_ratio > best_sharpe:
                    best_sharpe = sharpe_ratio
                    best_sma = (short_sma, long_sma)
                    best_trades = trade_count
                    best_sim_data = sim_data_eval.copy()

                self.short_sma = orig_short_sma
                self.long_sma = orig_long_sma

                sim_end_time = time.time()
                sim_time = sim_end_time - sim_start_time
                total_sim_time += sim_time
                
                completed += 1
                if completed % 100 == 0 or completed == total_combinations:
                    avg_sim_time = total_sim_time / completed
                    est_remaining = avg_sim_time * (total_combinations - completed)
                    print(
                        f"Progress: {completed}/{total_combinations} simulations completed ({(completed / total_combinations * 100):.1f}%)"
                        f" - Avg sim time: {avg_sim_time:.4f}s - Est. remaining: {est_remaining:.1f}s")

        if best_sim_data is not None:
            print("\n--- OPTIMIZATION SHARPE VERIFICATION ---")
            verify_split_idx = int(len(best_sim_data) * train_test_split)
            verify_returns = best_sim_data['Daily_Returns'].iloc[:verify_split_idx]
            
            if verify_returns.std() > 0:
                verify_sharpe = verify_returns.mean() / verify_returns.std() * np.sqrt(252)
                print(f"Optimization best Sharpe = {best_sharpe:.6f}")
                print(f"Verification Sharpe = {verify_sharpe:.6f}")
                print(f"Data points used: {len(best_sim_data)}")
                print(f"In-sample data points: {len(verify_returns)}")
            else:
                print("Cannot verify Sharpe (std = 0)")

        return best_sma, best_sharpe, best_trades, all_results

    def calculate_performance_metrics(self, data, strategy_name="Strategy", train_test_split=0.7):
        """
        Calculate detailed performance metrics for the strategy

        Parameters:
        data: DataFrame with strategy results
        strategy_name: Name suffix for the strategy columns
        train_test_split: Portion of data used for in-sample testing

        Returns:
        dict: Dictionary with performance metrics
        """
        # Create an explicit copy of the data to avoid SettingWithCopyWarning
        data_copy = data.copy()
        
        split_index = int(len(data_copy) * train_test_split)
        split_date = data_copy.index[split_index]

        daily_pnl = data_copy[f'Daily_PnL_{strategy_name}']
        returns_in_sample = daily_pnl.iloc[:split_index]
        returns_out_sample = daily_pnl.iloc[split_index:]

        sharpe_in_sample = returns_in_sample.mean() / returns_in_sample.std() * np.sqrt(
            252) if returns_in_sample.std() > 0 else 0
        sharpe_out_sample = returns_out_sample.mean() / returns_out_sample.std() * np.sqrt(
            252) if returns_out_sample.std() > 0 else 0
        sharpe_full = daily_pnl.mean() / daily_pnl.std() * np.sqrt(252) if daily_pnl.std() > 0 else 0

        print("\n--- PERFORMANCE METRICS SHARPE VERIFICATION ---")
        print(f"In-sample Sharpe = {sharpe_in_sample:.6f}")
        print(f"Data points used: {len(data_copy)}")
        print(f"In-sample data points: {len(returns_in_sample)}")
        print(f"Mean: {returns_in_sample.mean():.6f}, Std: {returns_in_sample.std():.6f}")

        position_changes = data_copy[f'Position_Change_{strategy_name}']
        total_trades = position_changes.sum()
        in_sample_trades = position_changes.iloc[:split_index].sum()
        out_sample_trades = position_changes.iloc[split_index:].sum()

        pnl_series = data_copy[f'Cumulative_PnL_{strategy_name}']
        
        # Calculate drawdown metrics on the copy
        data_copy['Peak'] = pnl_series.cummax()
        data_copy['Drawdown_Dollars'] = pnl_series - data_copy['Peak']
        max_drawdown_dollars = data_copy['Drawdown_Dollars'].min()

        total_pnl = data_copy[f'Cumulative_PnL_{strategy_name}'].iloc[-1]
        in_sample_pnl = data_copy[f'Daily_PnL_{strategy_name}'].iloc[:split_index].sum()
        out_sample_pnl = data_copy[f'Daily_PnL_{strategy_name}'].iloc[split_index:].sum()

        avg_position_size = data_copy[f'Position_Size_{strategy_name}'].mean()
        max_position_size = data_copy[f'Position_Size_{strategy_name}'].max()

        metrics = {
            'split_date': split_date,
            'total_pnl': total_pnl,
            'sharpe_full': sharpe_full,
            'sharpe_in_sample': sharpe_in_sample,
            'sharpe_out_sample': sharpe_out_sample,
            'max_drawdown_dollars': max_drawdown_dollars,
            'total_trades': total_trades,
            'in_sample_trades': in_sample_trades,
            'out_sample_trades': out_sample_trades,
            'in_sample_pnl': in_sample_pnl,
            'out_sample_pnl': out_sample_pnl,
            'avg_position_size': avg_position_size,
            'max_position_size': max_position_size
        }

        return metrics

## Configuration
Strategy parameters and settings

In [ ]:
# Configuration parameters
TICKER = 'MME'
START_DATE = '2014-01-01'
END_DATE = '2025-01-01'

# SMA range parameters
SMA_MIN = 10
SMA_MAX = 300
SMA_STEP = 5

# Data splitting ratio
TRAIN_TEST_SPLIT = 0.7

# ATR-based position sizing parameters
ATR_PERIOD = 30
TRADING_CAPITAL = 6000

## Main Execution
Execute the strategy with the defined configuration

In [ ]:
def main():
    # Start overall execution timer
    overall_start_time = time.time()
    
    # Setup paths using relative directories
    WORKING_DIR = "."
    DATA_DIR = os.path.join(WORKING_DIR, "data")

    # Define SYMBOL based on TICKER
    SYMBOL = TICKER.replace('=F', '')

    def save_parameters():
        parameters = {
            "big_point_value": big_point_value,
            "slippage": slippage,
            "capital": TRADING_CAPITAL,
            "atr_period": ATR_PERIOD
        }
        with open("parameters.json", "w") as file:
            json.dump(parameters, file)

    def get_slippage_from_excel(symbol, data_dir):
        excel_path = os.path.join(data_dir, "sessions_slippages.xlsx")
        
        if not os.path.exists(excel_path):
            raise FileNotFoundError(f"Slippage Excel file not found at {excel_path}")
        
        lookup_symbol = symbol.replace('=F', '')
        df = pd.read_excel(excel_path)
        
        if df.shape[1] < 4:
            raise ValueError(f"Excel file has fewer than 4 columns: {df.columns.tolist()}")
            
        df['SymbolUpper'] = df.iloc[:, 1].astype(str).str.upper()
        lookup_symbol_upper = lookup_symbol.upper()
        
        matching_rows = df[df['SymbolUpper'] == lookup_symbol_upper]
        
        if matching_rows.empty:
            raise ValueError(f"Symbol '{lookup_symbol}' not found in column B of Excel file")
            
        slippage_value = matching_rows.iloc[0, 3]
        
        if pd.isna(slippage_value) or not isinstance(slippage_value, (int, float)):
            raise ValueError(f"Invalid slippage value for symbol '{lookup_symbol}': {slippage_value}")
            
        print(f"Found slippage for {lookup_symbol} in column D: {slippage_value}")
        return slippage_value

    def find_futures_file(symbol, data_dir):
        """Find a data file for the specified futures symbol"""
        pattern = f"*@{symbol}_*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            pattern = f"*_@{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            pattern = f"*_{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            pattern = f"*@{symbol}*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            raise FileNotFoundError(f"No data file found for {symbol} in {data_dir}")
            
        return files[0]

    # Get symbol from the TICKER variable
    SYMBOL = TICKER.replace('=F', '')

    # Load the futures data file
    print("Loading data file...")
    load_start_time = time.time()
    
    data_files = glob.glob(os.path.join(DATA_DIR, f"*@{SYMBOL}*.dat"))
    if not data_files:
        data_files = glob.glob(os.path.join(DATA_DIR, f"*_{SYMBOL}_*.dat"))
    
    if not data_files:
        raise FileNotFoundError(f"No data file found for {SYMBOL} in {DATA_DIR}")
    
    all_data = read_ts_ohlcv_dat(data_files[0])
    load_end_time = time.time()
    load_time = load_end_time - load_start_time
    print(f"Data loaded successfully in {load_time:.2f} seconds! Number of items: {len(all_data)}")
    
    # Extract metadata and OHLCV data
    data_obj = all_data[0]
    tick_size = data_obj.big_point_value * data_obj.tick_size
    big_point_value = data_obj.big_point_value
    ohlc_data = data_obj.data.copy()

    # Fetch slippage value from Excel
    slippage_value = get_slippage_from_excel(TICKER, DATA_DIR)
    slippage = slippage_value
    print(f"Using slippage from Excel column D: {slippage}")

    # Save the parameters
    save_parameters()
    
    # Start timing data preparation
    prep_start_time = time.time()
    
    # Print information about the data
    print(f"\nSymbol: {data_obj.symbol}")
    print(f"Description: {data_obj.description}")
    print(f"Exchange: {data_obj.exchange}")
    print(f"Interval: {data_obj.interval_type} {data_obj.interval_span}")
    print(f"Tick size: {tick_size}")
    print(f"Big point value: {big_point_value}")
    print(f"Data shape: {ohlc_data.shape}")
    print(f"Date range: {ohlc_data['datetime'].min()} to {ohlc_data['datetime'].max()}")
    
    # Display the first few rows of data
    print("\nFirst few rows of OHLCV data:")
    print(ohlc_data.head())
    
    # Convert the OHLCV data to the expected format
    data = ohlc_data.rename(columns={
        'datetime': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    
    data.set_index('Date', inplace=True)
    
    # Add warm-up period for SMA calculation
    original_start_idx = None
    
    # Filter data to match the date range
    if START_DATE and END_DATE:
        warm_up_days = SMA_MAX + ATR_PERIOD + 50
        
        start_date = pd.to_datetime(START_DATE)
        end_date = pd.to_datetime(END_DATE)
        
        adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
        
        data = data[(data.index >= adjusted_start) & 
                    (data.index <= end_date)]
        
        if data.empty:
            raise ValueError(f"No data available for the specified date range: {START_DATE} to {END_DATE}")
            
        original_start_idx = data.index.get_indexer([start_date], method='nearest')[0]
        
        print(f"Loaded extended data with {warm_up_days} days warm-up period")
        print(f"Original date range: {START_DATE} to {END_DATE}")
        print(f"Adjusted date range: {adjusted_start.strftime('%Y-%m-%d')} to {END_DATE}")
        print(f"Original start index: {original_start_idx}")
    
    prep_end_time = time.time()
    prep_time = prep_end_time - prep_start_time
    print(f"Data preparation completed in {prep_time:.2f} seconds")
    
    # Define the range of SMA periods to test
    sma_range = range(SMA_MIN, SMA_MAX + 1, SMA_STEP)
    
    print(f"Optimizing SMA parameters using range from {SMA_MIN} to {SMA_MAX} with step {SMA_STEP}...")
    print(f"Trading with big point value from data: {big_point_value}")
    print(f"Using capital allocation: ${TRADING_CAPITAL:,} with ATR period: {ATR_PERIOD}")
    
    # Initialize the ATR-based strategy
    strategy = SMAStrategy(
        short_sma=0,
        long_sma=0,
        big_point_value=big_point_value,
        slippage=slippage,
        capital=TRADING_CAPITAL,
        atr_period=ATR_PERIOD
    )
    
    # Start optimization process
    print("\nStarting optimization process...")
    optimization_start_time = time.time()
    
    best_sma, best_sharpe, best_trades, all_results = strategy.optimize(
        data.copy(),
        sma_range,
        train_test_split=TRAIN_TEST_SPLIT,
        results_file='sma_all_results.txt',
        warm_up_idx=original_start_idx
    )
    
    optimization_end_time = time.time()
    optimization_time = optimization_end_time - optimization_start_time
    print(f"\nOptimization completed in {optimization_time:.2f} seconds ({optimization_time/60:.2f} minutes)")
    
    print(f"Optimal SMA parameters: Short = {best_sma[0]} days, Long = {best_sma[1]} days")
    print(f"In-sample Sharpe ratio = {best_sharpe:.4f}")
    print(f"Number of trades with optimal parameters = {best_trades}")
    print(f"Optimization results saved to 'sma_all_results.txt' for further analysis")
    
    # Update strategy with the best parameters
    strategy.short_sma = best_sma[0]
    strategy.long_sma = best_sma[1]
    
    # Apply the best strategy parameters
    print("\nApplying best strategy parameters...")
    apply_start_time = time.time()
    
    data = strategy.apply_strategy(data.copy())
    data_for_evaluation = data.copy()
    
    apply_end_time = time.time()
    apply_time = apply_end_time - apply_start_time
    print(f"Strategy application completed in {apply_time:.2f} seconds")
    
    # Start visualization process
    viz_start_time = time.time()
    
    if original_start_idx is not None:
        print("Trimming warm-up period for final evaluation and visualization...")
        data_for_evaluation = data.iloc[original_start_idx:]
        print(f"Original data length: {len(data)}, Evaluation data length: {len(data_for_evaluation)}")
    else:
        data_for_evaluation = data
    
    # Create visualization plots
    plt.figure(figsize=(14, 16))
    
    # Plot price and SMAs
    plt.subplot(3, 1, 1)
    plt.plot(data_for_evaluation.index, data_for_evaluation['Close'], label=f'{data_obj.symbol} Price', color='blue')
    plt.plot(data_for_evaluation.index, data_for_evaluation['SMA_Short_Strategy'], label=f'{best_sma[0]}-day SMA', color='orange')
    plt.plot(data_for_evaluation.index, data_for_evaluation['SMA_Long_Strategy'], label=f'{best_sma[1]}-day SMA', color='red')
    
    long_entries = (data_for_evaluation['Position_Dir_Strategy'] == 1) & data_for_evaluation['Position_Change_Strategy']
    short_entries = (data_for_evaluation['Position_Dir_Strategy'] == -1) & data_for_evaluation['Position_Change_Strategy']
    
    plt.scatter(data_for_evaluation.index[long_entries], data_for_evaluation.loc[long_entries, 'Close'], 
                color='green', marker='^', s=50, label='Long Entry')
    plt.scatter(data_for_evaluation.index[short_entries], data_for_evaluation.loc[short_entries, 'Close'], 
                color='red', marker='v', s=50, label='Short Entry')
    
    plt.legend()
    plt.title(f'{data_obj.symbol} with Optimized SMA Strategy ({best_sma[0]}, {best_sma[1]})')
    plt.grid(True)
    
    # Plot position size and ATR
    ax1 = plt.subplot(3, 1, 2)
    ax2 = ax1.twinx()
    
    ax1.plot(data_for_evaluation.index, data_for_evaluation['Position_Size_Strategy'], 
             label='Position Size (# Contracts)', color='purple')
    ax1.set_ylabel('Position Size (# Contracts)', color='purple')
    ax1.tick_params(axis='y', colors='purple')
    
    ax2.plot(data_for_evaluation.index, data_for_evaluation['ATR_Strategy'], 
             label=f'ATR ({ATR_PERIOD}-day)', color='orange')
    ax2.set_ylabel(f'ATR ({ATR_PERIOD}-day)', color='orange')
    ax2.tick_params(axis='y', colors='orange')
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.title(f'Position Sizing Based on {ATR_PERIOD}-day ATR')
    ax1.grid(True)
    
    # Plot performance
    plt.subplot(3, 1, 3)
    
    strategy_pnl_cumulative = data_for_evaluation['Cumulative_PnL_Strategy'] - data_for_evaluation['Cumulative_PnL_Strategy'].iloc[0]
    
    plt.plot(data_for_evaluation.index, strategy_pnl_cumulative, 
             label='Strategy P&L (full period)', color='green')
    
    split_index = int(len(data_for_evaluation) * TRAIN_TEST_SPLIT)
    
    plt.plot(data_for_evaluation.index[split_index:], strategy_pnl_cumulative.iloc[split_index:],
            label=f'Strategy P&L (last {int((1 - TRAIN_TEST_SPLIT) * 100)}% out-of-sample)', color='purple')
    
    plt.axvline(x=data_for_evaluation.index[split_index], color='black', linestyle='--',
                label=f'Train/Test Split ({int(TRAIN_TEST_SPLIT * 100)}%/{int((1 - TRAIN_TEST_SPLIT) * 100)}%)')
    plt.axhline(y=0.0, color='gray', linestyle='-', label='Break-even')
    
    plt.legend()
    plt.title('Strategy Performance (Dollar P&L)')
    plt.ylabel('P&L ($)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()  # Display the plot instead of saving it
    
    viz_end_time = time.time()
    viz_time = viz_end_time - viz_start_time
    print(f"Visualization completed in {viz_time:.2f} seconds")
    
    # Calculate performance metrics
    metrics_start_time = time.time()
    metrics = strategy.calculate_performance_metrics(
        data_for_evaluation,
        strategy_name="Strategy",
        train_test_split=TRAIN_TEST_SPLIT
    )
    metrics_end_time = time.time()
    metrics_time = metrics_end_time - metrics_start_time
    print(f"Performance metrics calculation completed in {metrics_time:.2f} seconds")
    
    # Calculate market performance for comparison
    market_cumulative_pnl = data_for_evaluation['Market_PnL_Strategy'].cumsum().iloc[-1]
    
    # Print summary statistics
    print("\n--- PERFORMANCE SUMMARY OF ATR-BASED SMA STRATEGY ---")
    print(f"Symbol: {data_obj.symbol}")
    print(f"Big Point Value (from data): {big_point_value}")
    print(f"ATR Period for Position Sizing: {ATR_PERIOD} days")
    print(f"Capital Allocation: ${TRADING_CAPITAL:,}")
    print(f"Average Position Size: {metrics['avg_position_size']:.2f} contracts")
    print(f"Maximum Position Size: {metrics['max_position_size']:.0f} contracts")
    print(f"Strategy Total P&L: ${metrics['total_pnl']:,.2f}")
    print(f"Market Buy & Hold P&L: ${market_cumulative_pnl:,.2f}")
    print(f"Outperformance: ${(metrics['total_pnl'] - market_cumulative_pnl):,.2f}")
    
    print("\n--- SHARPE RATIO COMPARISON VERIFICATION ---")
    print(f"Optimization in-sample Sharpe ratio: {best_sharpe:.6f}")
    print(f"Final in-sample Sharpe ratio: {metrics['sharpe_in_sample']:.6f}")
    print(f"Difference: {abs(best_sharpe - metrics['sharpe_in_sample']):.6f}")
    if abs(best_sharpe - metrics['sharpe_in_sample']) < 0.001:
        print("✓ SHARPE RATIOS MATCH (within 0.001 tolerance)")
    else:
        print("✗ SHARPE RATIOS DO NOT MATCH")
    
    print(f"Sharpe ratio (entire period, annualized): {metrics['sharpe_full']:.4f}")
    print(f"Sharpe ratio (in-sample, annualized): {metrics['sharpe_in_sample']:.4f}")
    print(f"Sharpe ratio (out-of-sample, annualized): {metrics['sharpe_out_sample']:.4f}")
    print(f"Maximum Drawdown: ${abs(metrics['max_drawdown_dollars']):,.2f}")
    
    print("\n--- TRADE COUNT SUMMARY ---")
    print(f"In-sample period trades: {metrics['in_sample_trades']}")
    print(f"Out-of-sample period trades: {metrics['out_sample_trades']}")
    print(f"Total trades: {metrics['total_trades']}")
    print(f"In-sample P&L: ${metrics['in_sample_pnl']:,.2f}")
    print(f"Out-of-sample P&L: ${metrics['out_sample_pnl']:,.2f}")
    
    print(f"\nBest parameters: Short SMA = {best_sma[0]}, Long SMA = {best_sma[1]}, Sharpe = {best_sharpe:.6f}, Trades = {best_trades}")
    
    # Calculate overall execution time
    overall_end_time = time.time()
    overall_time = overall_end_time - overall_start_time
    
    # Print timing summary
    print("\n--- EXECUTION TIME SUMMARY (Vectorized Implementation) ---")
    print(f"Data loading time: {load_time:.2f} seconds")
    print(f"Data preparation time: {prep_time:.2f} seconds")
    print(f"Optimization time: {optimization_time:.2f} seconds ({optimization_time/60:.2f} minutes)")
    print(f"Strategy application time: {apply_time:.2f} seconds")
    print(f"Visualization time: {viz_time:.2f} seconds")
    print(f"Metrics calculation time: {metrics_time:.2f} seconds")
    print(f"Total execution time: {overall_time:.2f} seconds ({overall_time/60:.2f} minutes)")
    
    print("\nAnalysis complete!")

    # After the existing strategy application and visualization, add the new analysis:
    print("\nStarting additional analysis...")
    
    # Run the basic analysis first
    data, best_short, best_long, best_sharpe, best_trades = analyze_sma_results()

    if data is None:
        print("Error: Failed to load or analyze SMA results data.")
        return

    print("\nProceeding with cluster analysis...")

    # Run the cluster analysis to get medoids
    X_filtered, medoids, top_medoids, centroids, max_sharpe_point = cluster_analysis()

    if X_filtered is None or medoids is None:
        print("Error: Cluster analysis failed.")
        return

    # Re-sort top_medoids to ensure they're in the right order
    if top_medoids is not None:
        print("Re-sorting top medoids by Sharpe ratio...")
        top_medoids = sorted(top_medoids, key=lambda x: float(x[2]), reverse=True)
        for idx, medoid in enumerate(top_medoids, 1):
            print(f"Verified Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    print("\nPlotting strategy performance...")

    # Plot strategy performance with the best parameters AND top medoids using ATR-based position sizing
    market_data = plot_strategy_performance(
        best_short, best_long, top_medoids, 
        big_point_value=big_point_value,
        slippage=slippage,
        capital=TRADING_CAPITAL,
        atr_period=ATR_PERIOD
    )
    
    # Run the bimonthly out-of-sample comparison between best Sharpe and top medoids
    if top_medoids and len(top_medoids) > 0:
        print("\nPerforming bimonthly out-of-sample comparison...")
        bimonthly_sharpe_df = bimonthly_out_of_sample_comparison(
            market_data, 
            best_short, 
            best_long, 
            top_medoids,  # Pass the entire top_medoids list
            big_point_value=big_point_value,
            slippage=slippage,
            capital=TRADING_CAPITAL,
            atr_period=ATR_PERIOD
        )
        
        # Calculate win percentage from bimonthly comparison
        if bimonthly_sharpe_df is not None:
            total_periods = len(bimonthly_sharpe_df)
            if total_periods > 0:
                rounded_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df['Best_sharpe_rounded'])
                win_percentage = (rounded_wins / total_periods) * 100
                
                # Save kmeans results to Excel
                save_to_excel(SYMBOL, win_percentage, top_medoids, best_short, best_long, is_kmeans=True)
    else:
        print("No top medoids found. Cannot run bimonthly comparison.")

    print("\nProceeding with hierarchical cluster analysis...")
    
    # Run the hierarchical cluster analysis
    X_filtered_h, medoids_h, top_medoids_h, max_sharpe_point_h, labels_h = hierarchical_cluster_analysis()

    if X_filtered_h is None or medoids_h is None:
        print("Error: Hierarchical cluster analysis failed.")
        exit(1)

    # Re-sort top_medoids to ensure they're in the right order
    if top_medoids_h is not None:
        print("Re-sorting hierarchical top medoids by Sharpe ratio...")
        top_medoids_h = sorted(top_medoids_h, key=lambda x: float(x[2]), reverse=True)
        for idx, medoid in enumerate(top_medoids_h, 1):
            print(f"Verified Hierarchical Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    print("\nPlotting hierarchical strategy performance...")

    # Plot strategy performance with hierarchical results
    market_data_h = plot_strategy_performance(
        best_short, best_long, top_medoids_h, 
        big_point_value=big_point_value,
        slippage=slippage,
        capital=TRADING_CAPITAL,
        atr_period=ATR_PERIOD,
        analysis_type="hierarchical"
    )
    
    # Run the bimonthly out-of-sample comparison for hierarchical
    if top_medoids_h and len(top_medoids_h) > 0:
        print("\nPerforming bimonthly out-of-sample comparison with hierarchical clustering...")
        bimonthly_sharpe_df_h = bimonthly_out_of_sample_comparison(
            market_data_h, 
            best_short, 
            best_long, 
            top_medoids_h,
            big_point_value=big_point_value,
            slippage=slippage,
            capital=TRADING_CAPITAL,
            atr_period=ATR_PERIOD,
            analysis_type="hierarchical"
        )
        
        # Calculate win percentage from bimonthly comparison
        if bimonthly_sharpe_df_h is not None:
            total_periods = len(bimonthly_sharpe_df_h)
            if total_periods > 0:
                rounded_wins = sum(bimonthly_sharpe_df_h['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df_h['Best_sharpe_rounded'])
                win_percentage = (rounded_wins / total_periods) * 100
                
                # Save hierarchical results to Excel
                save_to_excel(SYMBOL, win_percentage, top_medoids_h, best_short, best_long, is_kmeans=False)
    else:
        print("No hierarchical top medoids found. Cannot run bimonthly comparison.")

    print("\nAnalysis complete! All plots and result files have been saved.")

def compute_medoids(X, labels, valid_clusters):
    """Compute medoids for each cluster (point with minimum distance to all other points in cluster)"""
    medoids = []

    for cluster_id in valid_clusters:
        # Get points in this cluster
        cluster_points = X[labels == cluster_id]

        if len(cluster_points) == 0:
            continue

        # Calculate pairwise distances within cluster
        min_total_distance = float('inf')
        medoid = None

        for i, point1 in enumerate(cluster_points):
            total_distance = 0
            for j, point2 in enumerate(cluster_points):
                # Calculate Euclidean distance between points
                distance = np.sqrt(np.sum((point1 - point2) ** 2))
                total_distance += distance

            if total_distance < min_total_distance:
                min_total_distance = total_distance
                medoid = point1

        if medoid is not None:
            medoids.append(medoid)

    return medoids

def cluster_analysis():
    """
    Perform K-means clustering analysis on SMA optimization results
    """
    print(f"\n----- CLUSTER ANALYSIS -----")
    
    # First get the filtered data from analyze_sma_results
    data, _, _, _, _ = analyze_sma_results()
    
    if data is None:
        print("Error: Failed to load or analyze SMA results data.")
        return None, None, None, None, None

    # Convert filtered data to numpy array for clustering
    X_filtered = data[['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

    print(f"Using {len(X_filtered)} filtered data points for clustering")

    # Scale the data for clustering - using StandardScaler for each dimension
    scaler = StandardScaler()
    X_for_clustering = X_filtered[:, 0:3]  # Only use short_SMA, long_SMA, and sharpe_ratio for clustering
    X_scaled = scaler.fit_transform(X_for_clustering)

    # Print scaling info for verification
    print("\nScaled data information:")
    scaled_short = X_scaled[:, 0]
    scaled_long = X_scaled[:, 1]
    scaled_sharpe = X_scaled[:, 2]

    print(f"Scaled Short SMA range: {scaled_short.min():.4f} to {scaled_short.max():.4f}")
    print(f"Scaled Long SMA range: {scaled_long.min():.4f} to {scaled_long.max():.4f}")
    print(f"Scaled Sharpe ratio range: {scaled_sharpe.min():.4f} to {scaled_sharpe.max():.4f}")

    # Determine number of clusters
    print(f"Using default number of clusters: {DEFAULT_NUM_CLUSTERS}")
    k = DEFAULT_NUM_CLUSTERS

    # Apply KMeans clustering
    print(f"Performing KMeans clustering with k={k}...")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)

    # Get cluster labels
    labels = kmeans.labels_

    # Count elements per cluster
    unique_labels, counts = np.unique(labels, return_counts=True)
    cluster_sizes = dict(zip(unique_labels, counts))

    print("\nCluster sizes:")
    for cluster_id, size in cluster_sizes.items():
        print(f"Cluster {cluster_id}: {size} elements")

    # Filter clusters with enough elements
    valid_clusters = {i for i, count in cluster_sizes.items() if count >= MIN_ELEMENTS_PER_CLUSTER}
    if not valid_clusters:
        print(f"No clusters have at least {MIN_ELEMENTS_PER_CLUSTER} elements! Reducing threshold to 1...")
        valid_clusters = set(unique_labels)
    else:
        print(f"Using {len(valid_clusters)} clusters with at least {MIN_ELEMENTS_PER_CLUSTER} elements")

    filtered_indices = np.array([i in valid_clusters for i in labels])

    # Filter data to only include points in valid clusters
    X_valid = X_filtered[filtered_indices]
    labels_valid = labels[filtered_indices]

    # Compute medoids using the existing method that expects 4D data
    print("Computing medoids...")
    medoids = compute_medoids(X_valid, labels_valid, valid_clusters)

    # Compute centroids and inverse transform to get back to original scale
    centroids_scaled = kmeans.cluster_centers_
    centroids = scaler.inverse_transform(centroids_scaled)  # This will give us 3D centroids

    # Print raw centroids for debugging
    print("\nCluster Centroids (in original space):")
    for i, centroid in enumerate(centroids):
        if i in valid_clusters:  # Only show valid clusters
            print(f"Centroid {i}: Short SMA={centroid[0]:.2f}, Long SMA={centroid[1]:.2f}, "
                f"Sharpe={centroid[2]:.4f}")

    # Simply take top 5 medoids by Sharpe ratio
    top_medoids = sorted(medoids, key=lambda x: float(x[2]), reverse=True)[:5]
    
    # Debug print the sorted top medoids
    print("\nSELECTED TOP 5 MEDOIDS BY SHARPE RATIO:")
    for idx, medoid in enumerate(top_medoids, 1):
        print(f"Top {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
            f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    # Find max Sharpe ratio point overall
    max_sharpe_idx = np.argmax(data['sharpe_ratio'].values)
    max_sharpe_point = data.iloc[max_sharpe_idx][['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

    # Print results
    print("\n----- CLUSTERING RESULTS -----")
    print(f"Max Sharpe point: Short SMA={int(max_sharpe_point[0])}, Long SMA={int(max_sharpe_point[1])}, "
        f"Sharpe={max_sharpe_point[2]:.4f}, Trades={int(max_sharpe_point[3])}")

    print("\nTop 5 Medoids (by Sharpe ratio):")
    for idx, medoid in enumerate(top_medoids, 1):
        print(f"Top {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
            f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    # Create visualization with clustering results - pass the original labels and valid_clusters
    create_cluster_visualization(X_filtered, medoids, top_medoids, centroids, max_sharpe_point, 
                            labels=labels, valid_clusters=valid_clusters)

    return X_filtered, medoids, top_medoids, centroids, max_sharpe_point

def create_cluster_visualization(X_filtered_full, medoids, top_medoids, centroids, max_sharpe_point, labels=None, valid_clusters=None, analysis_type="kmeans"):
    """
    Create visualization of clustering results with both unfiltered and filtered heatmaps
    """
    print("Creating cluster visualization...")
    
    # Load the full dataset for the unfiltered heatmap
    data = pd.read_csv('sma_all_results.txt')
    
    # First plot: Unfiltered heatmap (only short_SMA < long_SMA)
    plt.figure(figsize=(12, 10))
    
    # Create a pivot table for the unfiltered heatmap
    unfiltered_heatmap = data.pivot_table(
        index='long_SMA',
        columns='short_SMA',
        values='sharpe_ratio',
        fill_value=np.nan
    )
    
    # Create mask for invalid combinations (where short_SMA >= long_SMA)
    unfiltered_mask = np.zeros_like(unfiltered_heatmap, dtype=bool)
    for i, long_sma in enumerate(unfiltered_heatmap.index):
        for j, short_sma in enumerate(unfiltered_heatmap.columns):
            if short_sma >= long_sma:
                unfiltered_mask[i, j] = True
    
    # Plot the unfiltered heatmap
    ax = sns.heatmap(
        unfiltered_heatmap,
        mask=unfiltered_mask,
        cmap='coolwarm',
        annot=False,
        fmt='.4f',
        linewidths=0,
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    
    ax.invert_yaxis()
    
    # Plot max Sharpe point on unfiltered heatmap
    try:
        best_x_pos = np.where(unfiltered_heatmap.columns == max_sharpe_point[0])[0][0] + 0.5
        best_y_pos = np.where(unfiltered_heatmap.index == max_sharpe_point[1])[0][0] + 0.5
        plt.scatter(best_x_pos, best_y_pos, marker='*', color='lime', s=200,
                    edgecolor='black', zorder=5)
    except IndexError:
        print(f"Warning: Max Sharpe point at ({max_sharpe_point[0]}, {max_sharpe_point[1]}) not found in unfiltered heatmap coordinates")
    
    # Create custom legend for unfiltered heatmap
    max_sharpe_handle = mlines.Line2D([], [], color='lime', marker='*', linestyle='None',
                                    markersize=15, markeredgecolor='black', label='Max Sharpe')
    
    plt.legend(handles=[max_sharpe_handle], loc='lower right')
    plt.title(f'Unfiltered {analysis_type} Heatmap (Only short_SMA < long_SMA)', pad=20)
    plt.xlabel('Short SMA (days)', fontsize=12)
    plt.ylabel('Long SMA (days)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    plt.close()
    
    # Second plot: Filtered heatmap with clustering results
    print("Creating filtered heatmap with clustering results...")
    
    # Create filtered dataframe from the filtered points that meet trade requirements
    filtered_df = pd.DataFrame(X_filtered_full, columns=['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades'])
    
    # Create a pivot table for the filtered heatmap
    heatmap_data = filtered_df.pivot_table(
        index='long_SMA',
        columns='short_SMA',
        values='sharpe_ratio',
        fill_value=np.nan
    )
    
    plt.figure(figsize=(12, 10))
    
    # Create mask for invalid combinations and NaN values
    mask = np.zeros_like(heatmap_data, dtype=bool)
    for i, long_sma in enumerate(heatmap_data.index):
        for j, short_sma in enumerate(heatmap_data.columns):
            if short_sma >= long_sma or np.isnan(heatmap_data.iloc[i, j]):
                mask[i, j] = True
    
    # Plot the filtered heatmap
    ax = sns.heatmap(
        heatmap_data,
        mask=mask,
        cmap='coolwarm',
        annot=False,
        fmt='.4f',
        linewidths=0,
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    
    ax.invert_yaxis()
    
    # Plot max Sharpe point
    try:
        best_x_pos = np.where(heatmap_data.columns == max_sharpe_point[0])[0][0] + 0.5
        best_y_pos = np.where(heatmap_data.index == max_sharpe_point[1])[0][0] + 0.5
        plt.scatter(best_x_pos, best_y_pos, marker='*', color='lime', s=200,
                    edgecolor='black', zorder=5)
    except IndexError:
        print(f"Warning: Max Sharpe point at ({max_sharpe_point[0]}, {max_sharpe_point[1]}) not found in filtered heatmap coordinates")
    
    # Plot medoids
    if medoids:
        for medoid in medoids:
            try:
                x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
                y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
                plt.scatter(x_pos, y_pos, marker='s', color='black', s=75, zorder=4)
            except IndexError:
                print(f"Warning: Medoid at ({medoid[0]}, {medoid[1]}) not found in filtered heatmap coordinates")
    
    # Plot top medoids
    if top_medoids:
        for medoid in top_medoids:
            try:
                x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
                y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
                plt.scatter(x_pos, y_pos, marker='D', color='purple', s=100, zorder=5)
            except IndexError:
                print(f"Warning: Top medoid at ({medoid[0]}, {medoid[1]}) not found in filtered heatmap coordinates")
    
    # Plot centroids
    print(f"Plotting centroids from valid clusters...")
    centroids_plotted = 0
    
    # If valid_clusters is not provided, assume all centroids are valid
    plot_centroids = centroids
    if valid_clusters is not None and labels is not None:
        plot_centroids = [centroids[i] for i in range(len(centroids)) if i in valid_clusters]
        print(f"Filtering centroids to only include valid clusters: {valid_clusters}")
    
    for i, centroid in enumerate(plot_centroids):
        short_sma = centroid[0]
        long_sma = centroid[1]
        
        print(f"Centroid {i}: raw values = ({short_sma}, {long_sma})")
        
        # First try exact values
        try:
            if (short_sma in heatmap_data.columns) and (long_sma in heatmap_data.index) and (short_sma < long_sma):
                x_pos = np.where(heatmap_data.columns == short_sma)[0][0] + 0.5
                y_pos = np.where(heatmap_data.index == long_sma)[0][0] + 0.5
                plt.scatter(x_pos, y_pos, marker='o', color='blue', s=75, zorder=4)
                centroids_plotted += 1
                continue
        except (IndexError, TypeError):
            pass
        
        # Try rounded values
        try:
            short_sma_rounded = int(round(short_sma))
            long_sma_rounded = int(round(long_sma))
            
            print(f"  Rounded: ({short_sma_rounded}, {long_sma_rounded})")
            
            if (short_sma_rounded in heatmap_data.columns) and (long_sma_rounded in heatmap_data.index) and (
                    short_sma_rounded < long_sma_rounded):
                x_pos = np.where(heatmap_data.columns == short_sma_rounded)[0][0] + 0.5
                y_pos = np.where(heatmap_data.index == long_sma_rounded)[0][0] + 0.5
                plt.scatter(x_pos, y_pos, marker='o', color='blue', s=75, zorder=4)
                centroids_plotted += 1
                continue
        except (IndexError, TypeError):
            pass
        
        # Try finding nearest valid point
        try:
            short_options = np.array(heatmap_data.columns)
            long_options = np.array(heatmap_data.index)
            
            short_idx = np.argmin(np.abs(short_options - short_sma))
            short_nearest = short_options[short_idx]
            
            long_idx = np.argmin(np.abs(long_options - long_sma))
            long_nearest = long_options[long_idx]
            
            print(f"  Nearest: ({short_nearest}, {long_nearest})")
            
            if short_nearest < long_nearest:
                x_pos = np.where(heatmap_data.columns == short_nearest)[0][0] + 0.5
                y_pos = np.where(heatmap_data.index == long_nearest)[0][0] + 0.5
                plt.scatter(x_pos, y_pos, marker='o', color='blue', s=75, zorder=4, alpha=0.7)
                centroids_plotted += 1
            else:
                print(f"  Invalid nearest parameters (short >= long): {short_nearest} >= {long_nearest}")
        except (IndexError, TypeError) as e:
            print(f"  Error finding nearest point: {e}")
    
    print(f"Successfully plotted {centroids_plotted} out of {len(plot_centroids)} centroids")
    
    # Create custom legend
    max_sharpe_handle = mlines.Line2D([], [], color='lime', marker='*', linestyle='None',
                                    markersize=15, markeredgecolor='black', label='Max Sharpe')
    # Use different legend labels based on analysis type
    if analysis_type == "hierarchical":
        cluster_handle = mlines.Line2D([], [], color='black', marker='s', linestyle='None',
                                    markersize=10, label='Clusters')
        top_cluster_handle = mlines.Line2D([], [], color='purple', marker='D', linestyle='None',
                                        markersize=10, label='Top 5 Clusters')
    else:
        cluster_handle = mlines.Line2D([], [], color='black', marker='s', linestyle='None',
                                    markersize=10, label='Medoids')
        top_cluster_handle = mlines.Line2D([], [], color='purple', marker='D', linestyle='None',
                                        markersize=10, label='Top 5 Medoids')
    centroid_handle = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                                    markersize=10, label='Centroids')
    
    # Add legend
    plt.legend(handles=[max_sharpe_handle, cluster_handle, top_cluster_handle, centroid_handle],
            loc='lower right')
    
    # Set labels and title
    plt.title(f'Filtered {analysis_type} Cluster Analysis', pad=20)
    plt.xlabel('Short SMA (days)', fontsize=12)
    plt.ylabel('Long SMA (days)', fontsize=12)
    
    # Rotate tick labels
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.show()
    plt.close()

def bimonthly_out_of_sample_comparison(data, best_short_sma, best_long_sma, top_medoids, min_sharpe=0.2, 
                                big_point_value=1, slippage=0, capital=TRADING_CAPITAL, atr_period=ATR_PERIOD,
                                analysis_type="kmeans"):  # Add analysis_type parameter
    """
    Compare the performance of the best Sharpe ratio strategy against a portfolio of top medoids
    on a bimonthly basis using out-of-sample data
    """
    print(f"\n----- BIMONTHLY OUT-OF-SAMPLE COMPARISON -----")
    print(f"Best Sharpe: ({best_short_sma}/{best_long_sma})")
    print(f"Using ATR-based position sizing (Capital: ${capital:,}, ATR Period: {atr_period})")
    
    # Handle the case where top_medoids is None
    if top_medoids is None:
        print("No medoids provided. Comparison cannot be performed.")
        return None
    
    # The top_medoids are already sorted by Sharpe ratio in cluster_analysis
    # Just print them for verification
    print("\nUSING TOP MEDOIDS (BY SHARPE RATIO):")
    for i, medoid in enumerate(top_medoids, 1):
        print(f"Medoid {i}: ({int(medoid[0])}/{int(medoid[1])}) - Sharpe: {float(medoid[2]):.4f}, Trades: {int(medoid[3])}")
    
    # Take at most 3 medoids and filter by minimum Sharpe
    filtered_medoids = []
    for i, m in enumerate(top_medoids[:3]):
        # Check if we can access the required elements
        try:
            # Extract Sharpe ratio and check if it meets the threshold
            short_sma = m[0]
            long_sma = m[1]
            sharpe = float(m[2])  # Convert to float to handle numpy types
            trades = m[3]
            
            if sharpe >= min_sharpe:
                filtered_medoids.append(m)
                print(f"Selected medoid {i+1} with Sharpe {sharpe:.4f}")
        except (IndexError, TypeError) as e:
            print(f"Error processing medoid: {e}")
    
    if not filtered_medoids:
        print(f"No medoids have a Sharpe ratio >= {min_sharpe}. Comparison cannot be performed.")
        return None
    
    print(f"Creating portfolio of {len(filtered_medoids)} medoids with Sharpe ratio >= {min_sharpe}:")
    for i, medoid in enumerate(filtered_medoids, 1):
        print(f"Final Medoid {i}: ({int(medoid[0])}/{int(medoid[1])}) - Sharpe: {float(medoid[2]):.4f}")
    
    # Create strategies
    strategies = {
        'Best': {'short_sma': best_short_sma, 'long_sma': best_long_sma}
    }
    
    # Add filtered medoids
    for i, medoid in enumerate(filtered_medoids, 1):
        strategies[f'Medoid_{i}'] = {'short_sma': int(medoid[0]), 'long_sma': int(medoid[1])}
    
    # Apply each strategy to the data
    for name, params in strategies.items():
        strategy = SMAStrategy(
            short_sma=params['short_sma'],
            long_sma=params['long_sma'],
            big_point_value=big_point_value,
            slippage=slippage,
            capital=capital,
            atr_period=atr_period
        )
        
        # Apply the strategy
        data = strategy.apply_strategy(
            data.copy(),
            strategy_name=name
        )
    
    # Get the out-of-sample split date
    split_index = int(len(data) * TRAIN_TEST_SPLIT)
    split_date = data.index[split_index]
    print(f"Out-of-sample period starts on: {split_date.strftime('%Y-%m-%d')}")
    
    # Get out-of-sample data
    oos_data = data.iloc[split_index:].copy()
    
    # Add a year and bimonthly period columns for grouping (each year has 6 bimonthly periods)
    oos_data['year'] = oos_data.index.year.astype(int)
    oos_data['bimonthly'] = ((oos_data.index.month - 1) // 2 + 1).astype(int)
    
    # Create simplified period labels with just the start month (YYYY-MM)
    oos_data['period_label'] = oos_data.apply(
        lambda row: f"{int(row['year'])}-{int((row['bimonthly'] - 1) * 2 + 1):02d}",
        axis=1
    )
    
    # Create a DataFrame to store bimonthly Sharpe ratios
    bimonthly_sharpe = []
    
    # Group by year and bimonthly period, calculate Sharpe ratio for each period
    for period_label, group in oos_data.groupby('period_label'):
        # Skip periods with too few trading days
        if len(group) < 10:
            continue
            
        # Create a bimonthly result entry
        year, start_month = period_label.split('-')
        year = int(year)
        start_month = int(start_month)
        
        bimonthly_result = {
            'period_label': period_label,
            'date': pd.Timestamp(year=year, month=start_month, day=15),  # Middle of first month in period
            'trading_days': len(group),
        }
        
        # Calculate Sharpe ratio for each strategy in this period
        for name in strategies.keys():
            # Get returns for this strategy in this period
            returns = group[f'Daily_PnL_{name}']
            
            # Calculate Sharpe ratio (annualized)
            if len(returns) > 1 and returns.std() > 0:
                sharpe = returns.mean() / returns.std() * np.sqrt(252)
            else:
                sharpe = 0
                
            bimonthly_result[f'{name}_sharpe'] = sharpe
            bimonthly_result[f'{name}_return'] = returns.sum()  # Total P&L for the period
        
        # Calculate the normalized average of medoid Sharpe ratios
        medoid_sharpes = [bimonthly_result[f'Medoid_{i}_sharpe'] for i in range(1, len(filtered_medoids) + 1)]
        bimonthly_result['Avg_Medoid_sharpe'] = sum(medoid_sharpes) / len(filtered_medoids)
        
        # Calculate the normalized average of medoid returns
        medoid_returns = [bimonthly_result[f'Medoid_{i}_return'] for i in range(1, len(filtered_medoids) + 1)]
        bimonthly_result['Avg_Medoid_return'] = sum(medoid_returns) / len(filtered_medoids)
        
        bimonthly_sharpe.append(bimonthly_result)
    
    # Convert to DataFrame
    bimonthly_sharpe_df = pd.DataFrame(bimonthly_sharpe)
    
    # Sort the DataFrame by date for proper chronological display
    if not bimonthly_sharpe_df.empty:
        bimonthly_sharpe_df = bimonthly_sharpe_df.sort_values('date')
    else:
        print(f"WARNING: No bimonthly periods found for {SYMBOL}. Cannot create chart.")
        return None
    
    # Add rounded values to dataframe for calculations
    bimonthly_sharpe_df['Best_sharpe_rounded'] = np.round(bimonthly_sharpe_df['Best_sharpe'], 2)
    bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] = np.round(bimonthly_sharpe_df['Avg_Medoid_sharpe'], 2)
    
    # Print detailed comparison of Sharpe ratios
    print("\nDetailed Sharpe ratio comparison by period:")
    print(f"{'Period':<12} | {'Best Sharpe':>12} | {'Medoid Portfolio':>16} | {'Difference':>12} | {'Portfolio Wins':<14}")
    print("-" * 80)
    
    for idx, row in bimonthly_sharpe_df.iterrows():
        period = row['period_label']
        best_sharpe = row['Best_sharpe']
        avg_medoid_sharpe = row['Avg_Medoid_sharpe']
        best_rounded = row['Best_sharpe_rounded']
        avg_medoid_rounded = row['Avg_Medoid_sharpe_rounded']
        
        diff = avg_medoid_sharpe - best_sharpe
        portfolio_wins = avg_medoid_sharpe > best_sharpe
        
        print(f"{period:<12} | {best_sharpe:12.6f} | {avg_medoid_sharpe:16.6f} | {diff:12.6f} | {portfolio_wins!s:<14}")
    
    # Calculate win rate using raw values
    portfolio_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe'] > bimonthly_sharpe_df['Best_sharpe'])
    total_periods = len(bimonthly_sharpe_df)
    win_percentage = (portfolio_wins / total_periods) * 100 if total_periods > 0 else 0
    
    # Calculate win rate using rounded values (for alternative comparison)
    rounded_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df['Best_sharpe_rounded'])
    rounded_win_percentage = (rounded_wins / total_periods) * 100 if total_periods > 0 else 0
    
    print(f"\nBimonthly periods analyzed: {total_periods}")
    print(f"Medoid Portfolio Wins: {portfolio_wins} of {total_periods} periods ({win_percentage:.2f}%)")
    print(f"Using rounded values (2 decimal places): {rounded_wins} of {total_periods} periods ({rounded_win_percentage:.2f}%)")
    
    # Create a bar plot to compare bimonthly Sharpe ratios
    plt.figure(figsize=(14, 8))
    
    # Set up x-axis dates
    x = np.arange(len(bimonthly_sharpe_df))
    width = 0.35  # Width of the bars
    
    # Create bars
    plt.bar(x - width/2, bimonthly_sharpe_df['Best_sharpe'], width, 
        label=f'Best Sharpe ({best_short_sma}/{best_long_sma})', color='blue')
    plt.bar(x + width/2, bimonthly_sharpe_df['Avg_Medoid_sharpe'], width, 
        label=f'Medoid Portfolio ({len(filtered_medoids)} strategies)', color='green')
    
    # Add a horizontal line at Sharpe = 0
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # Create medoid description for the title
    medoid_desc = ", ".join([f"({int(m[0])}/{int(m[1])})" for m in filtered_medoids])

    # Add analysis type to the title
    title_prefix = "K-means-Medoids" if analysis_type == "kmeans" else "Hierarchical"
    plt.title(f'{title_prefix} Bimonthly Out-of-Sample Comparison\nBest Sharpe vs Medoid Portfolio', pad=20)
    
    # Create legend with both strategies - moved to bottom to avoid overlap with title
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2,
            frameon=True, fancybox=True, framealpha=0.9, fontsize=10)
    
    # Add a text box with rounded win percentage instead of raw - moved to right side
    plt.annotate(f'Medoid Portfolio Win Rate: {rounded_win_percentage:.2f}%\n'
                f'({rounded_wins} out of {total_periods} periods)\n'
                f'Portfolio: {len(filtered_medoids)} medoids with Sharpe ≥ {min_sharpe}\n'
                f'ATR-Based Position Sizing (${capital:,}, {atr_period} days)',
                xy=(0.7, 0.95), xycoords='axes fraction',
                bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
                fontsize=12)
    
    # Use period labels for x-axis
    plt.xticks(x, [pd.to_datetime(label).strftime('%Y-%m') for label in bimonthly_sharpe_df['period_label']], rotation=45, ha='right')
    plt.xlabel('Bimonthly Period (Start Month)', fontsize=12)
    plt.ylabel('Sharpe Ratio (Annualized)', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Add extra space at the bottom for the legend
    plt.show()
    plt.close()

    return bimonthly_sharpe_df

def plot_strategy_performance(short_sma, long_sma, top_medoids=None, big_point_value=1, slippage=0, capital=TRADING_CAPITAL, atr_period=ATR_PERIOD, analysis_type="kmeans"):
    """
    Plot strategy performance including price, SMAs, and P&L for multiple strategies
    """
    print(f"\n----- PLOTTING STRATEGY PERFORMANCE -----")
    print(f"Using Short SMA: {short_sma}, Long SMA: {long_sma}")
    print(f"Trading with ATR-based position sizing (Capital: ${capital:,}, ATR Period: {atr_period})")
    if top_medoids:
        print(f"Including top {len(top_medoids)} medoids")

    # Load data from local file
    print(f"Loading {TICKER} data from local files...")
    data_file = find_futures_file(SYMBOL, DATA_DIR)
    if not data_file:
        print(f"Error: No data file found for {TICKER} in {DATA_DIR}")
        exit(1)

    print(f"Found data file: {os.path.basename(data_file)}")
    print(f"File size: {os.path.getsize(data_file)} bytes")

    # Load the data from local file
    all_data = read_ts_ohlcv_dat(data_file)
    data_obj = all_data[0]
    ohlc_data = data_obj.data.copy()

    # Convert the OHLCV data to the format expected by the strategy
    data = ohlc_data.rename(columns={
        'datetime': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    data.set_index('Date', inplace=True)

    # Add a warm-up period before the start date
    original_start_idx = None
    if START_DATE and END_DATE:
        # Calculate warm-up period for SMA calculation (longest SMA + buffer)
        warm_up_days = max(short_sma, long_sma) * 3  # Use 3x the longest SMA as warm-up
        
        # Convert dates to datetime
        start_date = pd.to_datetime(START_DATE)
        end_date = pd.to_datetime(END_DATE)
        
        # Adjust start date for warm-up
        adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
        
        # Filter with the extended date range
        extended_data = data[(data.index >= adjusted_start) & (data.index <= end_date)]
        
        # Store the index where the actual analysis should start
        if not extended_data.empty:
            # Find the closest index to our original start date
            original_start_idx = extended_data.index.get_indexer([start_date], method='nearest')[0]
        
        print(f"Added {warm_up_days} days warm-up period before {START_DATE}")
        data = extended_data

    # Create a dictionary to store results for each strategy
    strategies = {
        'Best': {'short_sma': short_sma, 'long_sma': long_sma}
    }

    # Add medoids in their original order, including original Sharpe and Trades
    if top_medoids:
        print("\nUSING THESE MEDOIDS (IN ORIGINAL ORDER):")
        for i, medoid in enumerate(top_medoids, 1):
            strategies[f'Medoid {i}'] = {
                'short_sma': int(medoid[0]),
                'long_sma': int(medoid[1]),
                'original_sharpe': float(medoid[2]),  # Store the original Sharpe ratio
                'original_trades': int(medoid[3])     # Store the original number of trades
            }
            print(f"Medoid {i}: SMA({int(medoid[0])}/{int(medoid[1])}) - Original Sharpe: {float(medoid[2]):.4f}, Trades: {int(medoid[3])}")

    # Apply the proper strategy for each parameter set
    for name, params in strategies.items():
        # Calculate centered SMAs directly
        data[f'SMA_Short_{name}'] = data['Close'].rolling(window=params['short_sma'], center=True).mean()
        data[f'SMA_Long_{name}'] = data['Close'].rolling(window=params['long_sma'], center=True).mean()
        
        # Create a strategy instance for each parameter set
        sma_strategy = SMAStrategy(
            short_sma=params['short_sma'],
            long_sma=params['long_sma'],
            big_point_value=big_point_value,
            slippage=slippage,
            capital=capital,
            atr_period=atr_period
        )

        # Apply the strategy
        data = sma_strategy.apply_strategy(
            data.copy(),
            strategy_name=name
        )

    # Trim data to the original date range if we added warm-up period
    if original_start_idx is not None:
        data_for_evaluation = data.iloc[original_start_idx:]
        print(f"Trimmed warm-up period. Original data length: {len(data)}, Evaluation data length: {len(data_for_evaluation)}")
    else:
        data_for_evaluation = data

    # Calculate split index for in-sample/out-of-sample using the trimmed data
    split_index = int(len(data_for_evaluation) * TRAIN_TEST_SPLIT)
    split_date = data_for_evaluation.index[split_index]

    # Add analysis type to the title
    title_prefix = "K-means-Medoids" if analysis_type == "kmeans" else "Hierarchical"
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), height_ratios=[1, 1.5])
    fig.suptitle(f'{title_prefix} Strategy Performance Analysis - {SYMBOL}', fontsize=16)
    
    # Plot price and SMA (first subplot)
    ax1.plot(data_for_evaluation.index, data_for_evaluation['Close'], label=f'{SYMBOL} Price', color='black', alpha=0.5)
    
    # Use the centered SMAs for plotting
    for name, params in strategies.items():
        if name == 'Best':
            ax1.plot(data_for_evaluation.index, data_for_evaluation[f'SMA_Short_{name}'], 
                    label=f'Short SMA ({params["short_sma"]})', color='orange')
            ax1.plot(data_for_evaluation.index, data_for_evaluation[f'SMA_Long_{name}'], 
                    label=f'Long SMA ({params["long_sma"]})', color='blue')
    
    # Mark the train/test split
    ax1.axvline(x=split_date, color='black', linestyle='--', alpha=0.7)
    
    ax1.legend(loc='upper left')
    ax1.set_title(f'{SYMBOL} Price and SMA Indicators')
    ax1.set_ylabel('Price')
    ax1.grid(True, alpha=0.3)

    # Plot cumulative P&L (second subplot)
    colors = ['blue', 'green', 'purple', 'orange', 'brown', 'pink']
    
    for i, (name, params) in enumerate(strategies.items()):
        color = colors[i % len(colors)]

        # Plot full period P&L
        ax2.plot(data_for_evaluation.index, data_for_evaluation[f'Cumulative_PnL_{name}'],
                label=f'{name} ({params["short_sma"]}/{params["long_sma"]})', color=color)

        # Plot out-of-sample portion with thicker line
        ax2.plot(data_for_evaluation.index[split_index:], data_for_evaluation[f'Cumulative_PnL_{name}'].iloc[split_index:],
                color=color, linewidth=2.5, alpha=0.7)

    ax2.axvline(x=split_date, color='black', linestyle='--',
                label=f'Train/Test Split ({int(TRAIN_TEST_SPLIT * 100)}%/{int((1 - TRAIN_TEST_SPLIT) * 100)}%)')
    ax2.axhline(y=0.0, color='gray', linestyle='-', alpha=0.5, label='Break-even')
    ax2.legend(loc='upper left')
    ax2.set_title('Strategy Cumulative P&L')
    ax2.set_ylabel('P&L ($)')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()
    plt.close()

    return data

def compute_hierarchical_medoids(X, labels, valid_clusters):
    """
    Compute medoids for each valid cluster from hierarchical clustering
    A medoid is the data point in a cluster that has the minimum average distance to all other points in the cluster

    Parameters:
    X: numpy array of shape (n_samples, n_features) - Original data points (not scaled)
    labels: numpy array of shape (n_samples,) - Cluster labels for each data point
    valid_clusters: set - Set of valid cluster IDs

    Returns:
    list of tuples - Each tuple contains (short_SMA, long_SMA, sharpe_ratio, trades) for each medoid
    """
    medoids = []
    for cluster_id in valid_clusters:
        cluster_indices = np.where(labels == cluster_id)[0]
        cluster_points = X[cluster_indices]

        if len(cluster_points) == 0:
            continue

        if len(cluster_points) == 1:
            medoids.append(tuple(cluster_points[0]))
            continue

        distances = cdist(cluster_points, cluster_points, metric='euclidean')
        total_distances = np.sum(distances, axis=1)
        medoid_idx = np.argmin(total_distances)
        medoid = cluster_points[medoid_idx]
        medoids.append(tuple(medoid))

        print(f"Cluster {cluster_id} medoid: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
            f"Sharpe={medoid[2]:.4f}, Trades={int(medoid[3])}")

    return medoids

def create_dendrogram(X_scaled, method='ward', figsize=(12, 8), color_threshold=None, truncate_mode=None,
                    p=10, save_path=None):
    """
    Create and display a dendrogram for hierarchical clustering
    """
    if len(X_scaled) > 1000:
        np.random.seed(42)
        sample_indices = np.random.choice(len(X_scaled), size=1000, replace=False)
        X_sample = X_scaled[sample_indices]
        print(f"Using a random sample of 1000 points for dendrogram creation (out of {len(X_scaled)} total points)")
    else:
        X_sample = X_scaled

    dist_matrix = pdist(X_sample, metric='euclidean')
    Z = shc.linkage(dist_matrix, method=method)

    print(f"\nDendrogram using {method} linkage method:")
    print(f"Number of data points: {len(X_sample)}")
    print(f"Cophenetic correlation: {shc.cophenet(Z, dist_matrix)[0]:.4f}")

    plt.figure(figsize=figsize)
    plt.gca().set_facecolor('white')
    plt.title(f'Hierarchical Clustering Dendrogram ({method} linkage)', fontsize=14)

    dendrogram = shc.dendrogram(
        Z,
        truncate_mode='lastp',
        p=30,
        leaf_rotation=90.,
        leaf_font_size=10.,
        show_contracted=True,
        color_threshold=color_threshold
    )

    plt.xlabel('Sample Index or Cluster Size', fontsize=12)
    plt.ylabel('Distance', fontsize=12)

    if color_threshold is not None:
        plt.axhline(y=color_threshold, color='crimson', linestyle='--',
                    label=f'Threshold: {color_threshold:.2f}')
        plt.legend()

    plt.tight_layout()
    plt.show()
    plt.close()

    return Z

def hierarchical_cluster_analysis(file_path='sma_all_results.txt', min_trades=MIN_TRADES, max_trades=MAX_TRADES,
                            min_elements_per_cluster=MIN_ELEMENTS_PER_CLUSTER):
    """
    Perform hierarchical clustering analysis on SMA optimization results
    """
    print(f"\n----- HIERARCHICAL CLUSTER ANALYSIS -----")
    print(f"Loading data from {file_path}...")

    df = pd.read_csv(file_path)
    X = df[['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

    X_filtered = X[(X[:, 0] < X[:, 1]) &
                (X[:, 3] >= min_trades) &
                (X[:, 3] <= max_trades)]

    if len(X_filtered) == 0:
        print(f"No data points meet the criteria after filtering!")
        return None, None, None, None, None

    print(f"Filtered data to {len(X_filtered)} points with {min_trades}-{max_trades} trades")

    short_sma_values = X_filtered[:, 0]
    long_sma_values = X_filtered[:, 1]
    sharpe_values = X_filtered[:, 2]
    trades_values = X_filtered[:, 3]

    print(f"Short SMA range: {short_sma_values.min()} to {short_sma_values.max()}")
    print(f"Long SMA range: {long_sma_values.min()} to {long_sma_values.max()}")
    print(f"Sharpe ratio range: {sharpe_values.min():.4f} to {sharpe_values.max():.4f}")
    print(f"Trades range: {trades_values.min()} to {trades_values.max()}")

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_filtered)

    print("\nScaled data information:")
    scaled_short = X_scaled[:, 0]
    scaled_long = X_scaled[:, 1]
    scaled_sharpe = X_scaled[:, 2]
    scaled_trades = X_scaled[:, 3]

    print(f"Scaled Short SMA range: {scaled_short.min():.4f} to {scaled_short.max():.4f}")
    print(f"Scaled Long SMA range: {scaled_long.min():.4f} to {scaled_long.max():.4f}")
    print(f"Scaled Sharpe ratio range: {scaled_sharpe.min():.4f} to {scaled_sharpe.max():.4f}")
    print(f"Scaled Trades range: {scaled_trades.min():.4f} to {scaled_trades.max():.4f}")

    print("\nCreating dendrogram to visualize hierarchical structure...")
    linkage_method = 'ward'
    Z = create_dendrogram(X_scaled, method=linkage_method, figsize=(12, 8))

    print(f"Using default number of clusters based on dendrogram: {DEFAULT_NUM_CLUSTERS}")
    k = DEFAULT_NUM_CLUSTERS

    print(f"Performing hierarchical clustering with {k} clusters using {linkage_method} linkage...")
    hierarchical = AgglomerativeClustering(
        n_clusters=k,
        linkage=linkage_method
    )

    labels = hierarchical.fit_predict(X_scaled)
    unique_labels, counts = np.unique(labels, return_counts=True)
    cluster_sizes = dict(zip(unique_labels, counts))

    print("\nCluster sizes:")
    for cluster_id, size in cluster_sizes.items():
        print(f"Cluster {cluster_id}: {size} elements")

    valid_clusters = {i for i, count in cluster_sizes.items() if count >= min_elements_per_cluster}
    if not valid_clusters:
        print(f"No clusters have at least {min_elements_per_cluster} elements! Reducing threshold to 1...")
        valid_clusters = set(unique_labels)
    else:
        print(f"Using {len(valid_clusters)} clusters with at least {min_elements_per_cluster} elements")

    filtered_indices = np.array([i in valid_clusters for i in labels])
    X_valid = X_filtered[filtered_indices]
    labels_valid = labels[filtered_indices]

    print("Computing medoids for each cluster...")
    medoids = compute_hierarchical_medoids(X_valid, labels_valid, valid_clusters)

    max_sharpe_idx = np.argmax(df['sharpe_ratio'].values)
    max_sharpe_point = df.iloc[max_sharpe_idx][['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

    medoids_sorted = sorted(medoids, key=lambda x: float(x[2]), reverse=True)
    top_medoids = medoids_sorted[:5]

    print("\nDEBUG - SORTED MEDOIDS BY SHARPE RATIO:")
    for idx, medoid in enumerate(medoids_sorted[:5], 1):
        print(f"Sorted Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
            f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    print("\n----- HIERARCHICAL CLUSTERING RESULTS -----")
    print(f"Max Sharpe point: Short SMA={int(max_sharpe_point[0])}, Long SMA={int(max_sharpe_point[1])}, "
        f"Sharpe={max_sharpe_point[2]:.4f}, Trades={int(max_sharpe_point[3])}")

    print("\nTop 5 Medoids (by Sharpe ratio):")
    for idx, medoid in enumerate(top_medoids, 1):
        print(f"Top {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
            f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    # Create visualization with clustering results
    create_hierarchical_cluster_visualization(X_filtered, medoids, top_medoids, max_sharpe_point, labels, analysis_type="hierarchical")

    clustering_results = []
    for idx, medoid in enumerate(medoids_sorted):
        clustering_results.append({
            'Rank': idx + 1,
            'Short_SMA': int(medoid[0]),
            'Long_SMA': int(medoid[1]),
            'Sharpe': medoid[2],
            'Trades': int(medoid[3])
        })
    

    return X_filtered, medoids, top_medoids, max_sharpe_point, labels

def create_hierarchical_cluster_visualization(X, medoids, top_medoids, max_sharpe_point, labels, analysis_type="hierarchical"):
    """Create visualization of hierarchical clustering results with heatmap plot"""
    print("Creating hierarchical cluster visualization...")
    
    # First filter by valid clusters
    valid_cluster_ids = set([labels[np.where((X[:, 0] == medoid[0]) & (X[:, 1] == medoid[1]))[0][0]] for medoid in medoids])
    print(f"Using {len(valid_cluster_ids)} valid clusters with medoids")
    
    # Filter by valid clusters
    valid_indices = np.array([label in valid_cluster_ids for label in labels])
    X_valid = X[valid_indices]
    labels_valid = labels[valid_indices]
    
    # Additional filtering: ensure short_SMA < long_SMA and trades within range
    X_filtered = X_valid[
        (X_valid[:, 0] < X_valid[:, 1]) &  # short_SMA < long_SMA
        (X_valid[:, 3] >= MIN_TRADES) &     # trades >= min_trades
        (X_valid[:, 3] <= MAX_TRADES)       # trades <= max_trades
    ]
    
    print(f"Filtering to {len(X_filtered)} points from valid clusters")
    
    # Create DataFrame for heatmap
    filtered_df = pd.DataFrame(X_filtered, columns=['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades'])
    
    # Load the full dataset to create complete grid
    full_data = pd.read_csv('sma_all_results.txt')
    
    # Create a complete pivot table with all possible combinations
    heatmap_data = pd.pivot_table(
        full_data,
        index='long_SMA',
        columns='short_SMA',
        values='sharpe_ratio',
        fill_value=np.nan
    )
    
    # Now overlay our filtered data
    filtered_pivot = pd.pivot_table(
        filtered_df,
        index='long_SMA',
        columns='short_SMA',
        values='sharpe_ratio',
        fill_value=np.nan
    )
    
    # Update heatmap_data with filtered values
    for long_sma in filtered_pivot.index:
        for short_sma in filtered_pivot.columns:
            if not np.isnan(filtered_pivot.loc[long_sma, short_sma]):
                heatmap_data.loc[long_sma, short_sma] = filtered_pivot.loc[long_sma, short_sma]
    
    # Plot heatmap
    plt.figure(figsize=(12, 10))
    
    # Create mask for invalid combinations and NaN values
    mask = np.zeros_like(heatmap_data, dtype=bool)
    for i, long_sma in enumerate(heatmap_data.index):
        for j, short_sma in enumerate(heatmap_data.columns):
            # Only mask if short_SMA >= long_SMA (invalid combination)
            if short_sma >= long_sma:
                mask[i, j] = True

    # Plot heatmap
    ax = sns.heatmap(
        heatmap_data,
        mask=mask,
        cmap='coolwarm',
        annot=False,
        fmt='.4f',
        linewidths=0,
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    
    ax.invert_yaxis()
    
    # Plot points on the heatmap
    for medoid in medoids:
        try:
            x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
            y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
            plt.scatter(x_pos, y_pos, marker='s', color='black', s=75, zorder=4)
        except IndexError:
            print(f"Warning: Medoid at ({medoid[0]}, {medoid[1]}) not found in heatmap coordinates")

    for medoid in top_medoids:
        try:
            x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
            y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
            plt.scatter(x_pos, y_pos, marker='D', color='purple', s=100, zorder=5)
        except IndexError:
            print(f"Warning: Top medoid at ({medoid[0]}, {medoid[1]}) not found in heatmap coordinates")

    try:
        best_x_pos = np.where(heatmap_data.columns == max_sharpe_point[0])[0][0] + 0.5
        best_y_pos = np.where(heatmap_data.index == max_sharpe_point[1])[0][0] + 0.5
        plt.scatter(best_x_pos, best_y_pos, marker='*', color='lime', s=200,
                edgecolor='black', zorder=5)
    except IndexError:
        print(f"Warning: Max Sharpe point at ({max_sharpe_point[0]}, {max_sharpe_point[1]}) not found in heatmap coordinates")

    # Create custom legend
    max_sharpe_handle = mlines.Line2D([], [], color='lime', marker='*', linestyle='None',
                                    markersize=15, markeredgecolor='black', label='Max Sharpe')
    cluster_handle = mlines.Line2D([], [], color='black', marker='s', linestyle='None',
                                markersize=10, label='Clusters')
    top_cluster_handle = mlines.Line2D([], [], color='purple', marker='D', linestyle='None',
                                    markersize=10, label='Top 5 Clusters')
    
    # Add legend
    plt.legend(handles=[max_sharpe_handle, cluster_handle, top_cluster_handle],
            loc='lower right')
    
    # Add labels and title
    plt.title('Hierarchical Clustering Analysis (Sharpe Ratio)', fontsize=14)
    plt.xlabel('Short SMA (days)', fontsize=12)
    plt.ylabel('Long SMA (days)', fontsize=12)

    # Rotate tick labels
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.tight_layout()
    plt.show()
    plt.close()

def save_to_excel(symbol, win_percentage, medoids, best_short_sma, best_long_sma, is_kmeans=True):
    """Save clustering results to Excel file"""
    try:
        # Path to the Excel file
        excel_file = r"C:\Users\Admin\Documents\darbas\Results.xlsx"
        
        # Check if file exists
        if not os.path.exists(excel_file):
            print(f"Excel file not found at: {excel_file}")
            return
            
        print(f"Updating Excel file with {'K-means' if is_kmeans else 'Hierarchical'} results for {symbol}...")
        
        # Load the workbook
        wb = openpyxl.load_workbook(excel_file)
        
        # Get the active sheet
        sheet = wb.active
        
        # Find the row with the ticker symbol or the first empty row
        row = 3  # Start from row 3 (assuming rows 1-2 have headers)
        ticker_row = None
        
        while True:
            cell_value = sheet.cell(row=row, column=1).value
            if cell_value == symbol:
                # Found the ticker symbol
                ticker_row = row
                break
            elif cell_value is None:
                # Found an empty row
                ticker_row = row
                # Write the ticker symbol in column A
                sheet.cell(row=ticker_row, column=1).value = symbol
                break
            row += 1
        
        # Round the win percentage to one decimal place
        rounded_win_percentage = round(win_percentage, 1)
        
        # Write the win percentage in column B (K-means) or C (Hierarchical)
        column = 2 if is_kmeans else 3
        sheet.cell(row=ticker_row, column=column).value = rounded_win_percentage
        
        # Write the medoid parameters in the respective cluster columns
        # K-means: E=5, F=6, G=7 for Cluster1, Cluster2, Cluster3
        # Hierarchical: I=9, J=10, K=11 for Cluster1, Cluster2, Cluster3
        start_column = 5 if is_kmeans else 9
        
        for i, medoid in enumerate(medoids):
            if i >= 3:  # Only use up to 3 clusters
                break
                
            # Calculate column index
            column_idx = start_column + i
            
            # Format as "short/long" (e.g., "5/20")
            param_value = f"{int(medoid[0])}/{int(medoid[1])}"
            
            # Write to Excel
            sheet.cell(row=ticker_row, column=column_idx).value = param_value
        
        # Write the best Sharpe parameters in column M (13)
        best_sharpe_params = f"{best_short_sma}/{best_long_sma}"
        sheet.cell(row=ticker_row, column=13).value = best_sharpe_params
        
        # Save the workbook
        wb.save(excel_file)
        
        print(f"Excel file updated successfully. Added {symbol} with {'K-means' if is_kmeans else 'Hierarchical'} win rate {rounded_win_percentage}% in row {ticker_row}")
        print(f"Added best Sharpe parameters {best_sharpe_params} in column M")
        
    except Exception as e:
        print(f"Error updating Excel file: {e}")

In [ ]:
if __name__ == "__main__":
    import sys
    import importlib

    def get_slippage_from_excel(symbol, data_dir):
        excel_path = os.path.join(data_dir, "sessions_slippages.xlsx")
        
        if not os.path.exists(excel_path):
            raise FileNotFoundError(f"Slippage Excel file not found at {excel_path}")
        
        lookup_symbol = symbol.replace('=F', '')
        df = pd.read_excel(excel_path)
        
        if df.shape[1] < 4:
            raise ValueError(f"Excel file has fewer than 4 columns: {df.columns.tolist()}")
            
        df['SymbolUpper'] = df.iloc[:, 1].astype(str).str.upper()
        lookup_symbol_upper = lookup_symbol.upper()
        
        matching_rows = df[df['SymbolUpper'] == lookup_symbol_upper]
        
        if matching_rows.empty:
            raise ValueError(f"Symbol '{lookup_symbol}' not found in column B of Excel file")
            
        slippage_value = matching_rows.iloc[0, 3]
        
        if pd.isna(slippage_value) or not isinstance(slippage_value, (int, float)):
            raise ValueError(f"Invalid slippage value for symbol '{lookup_symbol}': {slippage_value}")
            
        print(f"Found slippage for {lookup_symbol} in column D: {slippage_value}")
        return slippage_value

    # List of all underlyings to process
    underlyings = [
        "AD", "BO", "BP", "BRN", "C", "CC", "CD", "CL", "CT", "DX", 
        "EC", "EMD", "ES", "FC", "FDAX", "FESX", "FGBL", "FGBM", "FGBX", 
        "FV", "GC", "HG", "HO", "JY", "KC", "KW", "LC", "LH", "LJ", 
        "LZ", "MME", "MP1", "NE1", "NG", "NK", "NQ", "PA", "PL", 
        "RB", "RTY", "S", "SB", "SF", "SI", "SM", "TEN", "TY", 
        "TU", "UB", "ULS", "US", "VX", "W", "WBS"
    ]

    # Process each underlying
    for current_ticker in underlyings:
        print(f"\n{'='*80}")
        print(f"Processing {current_ticker}")
        print(f"{'='*80}\n")

        try:
            # Override the TICKER from input.py
            input.TICKER = current_ticker
            TICKER = current_ticker  # Update local TICKER variable

            # Start overall execution timer
            overall_start_time = time.time()
            
            # Setup paths using relative directories
            WORKING_DIR = "."
            DATA_DIR = os.path.join(WORKING_DIR, "data")

            # Define SYMBOL based on TICKER
            SYMBOL = TICKER.replace('=F', '')

            # Load the futures data file
            print("Loading data file...")
            load_start_time = time.time()
            
            data_files = glob.glob(os.path.join(DATA_DIR, f"*@{SYMBOL}*.dat"))
            if not data_files:
                data_files = glob.glob(os.path.join(DATA_DIR, f"*_{SYMBOL}_*.dat"))
            
            if not data_files:
                raise FileNotFoundError(f"No data file found for {SYMBOL} in {DATA_DIR}")
            
            all_data = read_ts_ohlcv_dat(data_files[0])
            load_end_time = time.time()
            load_time = load_end_time - load_start_time
            print(f"Data loaded successfully in {load_time:.2f} seconds! Number of items: {len(all_data)}")
            
            # Extract metadata and OHLCV data
            data_obj = all_data[0]
            tick_size = data_obj.big_point_value * data_obj.tick_size
            big_point_value = data_obj.big_point_value
            ohlc_data = data_obj.data.copy()

            # Fetch slippage value from Excel
            slippage_value = get_slippage_from_excel(TICKER, DATA_DIR)
            slippage = slippage_value
            print(f"Using slippage from Excel column D: {slippage}")

            # Save parameters
            parameters = {
                "big_point_value": big_point_value,
                "slippage": slippage,
                "capital": TRADING_CAPITAL,
                "atr_period": ATR_PERIOD
            }
            with open("parameters.json", "w") as file:
                json.dump(parameters, file)
            
            # Print information about the data
            print(f"\nSymbol: {data_obj.symbol}")
            print(f"Description: {data_obj.description}")
            print(f"Exchange: {data_obj.exchange}")
            print(f"Interval: {data_obj.interval_type} {data_obj.interval_span}")
            print(f"Tick size: {tick_size}")
            print(f"Big point value: {big_point_value}")
            print(f"Data shape: {ohlc_data.shape}")
            print(f"Date range: {ohlc_data['datetime'].min()} to {ohlc_data['datetime'].max()}")
            
            # Convert the OHLCV data to the expected format
            data = ohlc_data.rename(columns={
                'datetime': 'Date',
                'open': 'Open',
                'high': 'High',
                'low': 'Low',
                'close': 'Close',
                'volume': 'Volume'
            })
            
            data.set_index('Date', inplace=True)
            
            # Add warm-up period for SMA calculation
            original_start_idx = None
            
            # Filter data to match the date range
            if START_DATE and END_DATE:
                warm_up_days = SMA_MAX + ATR_PERIOD + 50
                
                start_date = pd.to_datetime(START_DATE)
                end_date = pd.to_datetime(END_DATE)
                
                adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
                
                data = data[(data.index >= adjusted_start) & 
                            (data.index <= end_date)]
                
                if data.empty:
                    raise ValueError(f"No data available for the specified date range: {START_DATE} to {END_DATE}")
                    
                original_start_idx = data.index.get_indexer([start_date], method='nearest')[0]
                
                print(f"Loaded extended data with {warm_up_days} days warm-up period")
                print(f"Original date range: {START_DATE} to {END_DATE}")
                print(f"Adjusted date range: {adjusted_start.strftime('%Y-%m-%d')} to {END_DATE}")
                print(f"Original start index: {original_start_idx}")
            
            # Define the range of SMA periods to test
            sma_range = range(SMA_MIN, SMA_MAX + 1, SMA_STEP)
            
            print(f"Optimizing SMA parameters using range from {SMA_MIN} to {SMA_MAX} with step {SMA_STEP}...")
            print(f"Trading with big point value from data: {big_point_value}")
            print(f"Using capital allocation: ${TRADING_CAPITAL:,} with ATR period: {ATR_PERIOD}")
            
            # Initialize the ATR-based strategy
            strategy = SMAStrategy(
                short_sma=0,
                long_sma=0,
                big_point_value=big_point_value,
                slippage=slippage,
                capital=TRADING_CAPITAL,
                atr_period=ATR_PERIOD
            )
            
            # Start optimization process
            print("\nStarting optimization process...")
            optimization_start_time = time.time()
            
            best_sma, best_sharpe, best_trades, all_results = strategy.optimize(
                data.copy(),
                sma_range,
                train_test_split=TRAIN_TEST_SPLIT,
                results_file='sma_all_results.txt',
                warm_up_idx=original_start_idx
            )
            
            optimization_end_time = time.time()
            optimization_time = optimization_end_time - optimization_start_time
            print(f"\nOptimization completed in {optimization_time:.2f} seconds ({optimization_time/60:.2f} minutes)")
            
            print(f"Optimal SMA parameters: Short = {best_sma[0]} days, Long = {best_sma[1]} days")
            print(f"In-sample Sharpe ratio = {best_sharpe:.4f}")
            print(f"Number of trades with optimal parameters = {best_trades}")
            print(f"Optimization results saved to 'sma_all_results.txt' for further analysis")

            # Apply the best strategy parameters
            print("\nApplying best strategy parameters...")
            strategy.short_sma = best_sma[0]
            strategy.long_sma = best_sma[1]
            
            data = strategy.apply_strategy(data.copy())
            data_for_evaluation = data.copy()
            
            if original_start_idx is not None:
                print("Trimming warm-up period for final evaluation and visualization...")
                data_for_evaluation = data.iloc[original_start_idx:]
                print(f"Original data length: {len(data)}, Evaluation data length: {len(data_for_evaluation)}")
            else:
                data_for_evaluation = data
            
            # Create visualization plots
            plt.figure(figsize=(14, 16))
            
            # Plot price and SMAs
            plt.subplot(3, 1, 1)
            plt.plot(data_for_evaluation.index, data_for_evaluation['Close'], label=f'{data_obj.symbol} Price', color='blue')
            plt.plot(data_for_evaluation.index, data_for_evaluation['SMA_Short_Strategy'], label=f'{best_sma[0]}-day SMA', color='orange')
            plt.plot(data_for_evaluation.index, data_for_evaluation['SMA_Long_Strategy'], label=f'{best_sma[1]}-day SMA', color='red')
            
            long_entries = (data_for_evaluation['Position_Dir_Strategy'] == 1) & data_for_evaluation['Position_Change_Strategy']
            short_entries = (data_for_evaluation['Position_Dir_Strategy'] == -1) & data_for_evaluation['Position_Change_Strategy']
            
            plt.scatter(data_for_evaluation.index[long_entries], data_for_evaluation.loc[long_entries, 'Close'], 
                        color='green', marker='^', s=50, label='Long Entry')
            plt.scatter(data_for_evaluation.index[short_entries], data_for_evaluation.loc[short_entries, 'Close'], 
                        color='red', marker='v', s=50, label='Short Entry')
            
            plt.legend()
            plt.title(f'{data_obj.symbol} with Optimized SMA Strategy ({best_sma[0]}, {best_sma[1]})')
            plt.grid(True)
            
            # Plot position size and ATR
            ax1 = plt.subplot(3, 1, 2)
            ax2 = ax1.twinx()
            
            ax1.plot(data_for_evaluation.index, data_for_evaluation['Position_Size_Strategy'], 
                     label='Position Size (# Contracts)', color='purple')
            ax1.set_ylabel('Position Size (# Contracts)', color='purple')
            ax1.tick_params(axis='y', colors='purple')
            
            ax2.plot(data_for_evaluation.index, data_for_evaluation['ATR_Strategy'], 
                     label=f'ATR ({ATR_PERIOD}-day)', color='orange')
            ax2.set_ylabel(f'ATR ({ATR_PERIOD}-day)', color='orange')
            ax2.tick_params(axis='y', colors='orange')
            
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
            
            plt.title(f'Position Sizing Based on {ATR_PERIOD}-day ATR')
            ax1.grid(True)
            
            # Plot performance
            plt.subplot(3, 1, 3)
            
            strategy_pnl_cumulative = data_for_evaluation['Cumulative_PnL_Strategy'] - data_for_evaluation['Cumulative_PnL_Strategy'].iloc[0]
            
            plt.plot(data_for_evaluation.index, strategy_pnl_cumulative, 
                     label='Strategy P&L (full period)', color='green')
            
            split_index = int(len(data_for_evaluation) * TRAIN_TEST_SPLIT)
            
            plt.plot(data_for_evaluation.index[split_index:], strategy_pnl_cumulative.iloc[split_index:],
                    label=f'Strategy P&L (last {int((1 - TRAIN_TEST_SPLIT) * 100)}% out-of-sample)', color='purple')
            
            plt.axvline(x=data_for_evaluation.index[split_index], color='black', linestyle='--',
                        label=f'Train/Test Split ({int(TRAIN_TEST_SPLIT * 100)}%/{int((1 - TRAIN_TEST_SPLIT) * 100)}%)')
            plt.axhline(y=0.0, color='gray', linestyle='-', label='Break-even')
            
            plt.legend()
            plt.title('Strategy Performance (Dollar P&L)')
            plt.ylabel('P&L ($)')
            plt.grid(True)
            
            plt.tight_layout()
            plt.show()

            print("\nStarting analysis of optimization results...")

            # Run the basic analysis first to get best parameters
            data, best_short, best_long, best_sharpe, best_trades = analyze_sma_results()

            if data is None:
                print("Error: Failed to load or analyze SMA results data.")
                continue

            print("\nProceeding with K-means cluster analysis...")
            
            # Run the K-means cluster analysis to get medoids
            X_filtered, medoids, top_medoids, centroids, max_sharpe_point = cluster_analysis()

            if X_filtered is None or medoids is None:
                print("Error: K-means cluster analysis failed.")
                continue

            # Re-sort top_medoids to ensure they're in the right order
            if top_medoids is not None:
                print("Re-sorting K-means top medoids by Sharpe ratio...")
                top_medoids = sorted(top_medoids, key=lambda x: float(x[2]), reverse=True)
                for idx, medoid in enumerate(top_medoids, 1):
                    print(f"Verified K-means Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                        f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

            print("\nPlotting K-means strategy performance...")

            # Plot strategy performance with K-means results
            market_data = plot_strategy_performance(
                best_short, best_long, top_medoids, 
                big_point_value=big_point_value,
                slippage=slippage,
                capital=TRADING_CAPITAL,
                atr_period=ATR_PERIOD,
                analysis_type="kmeans"
            )
            
            # Run the bimonthly out-of-sample comparison for K-means
            if top_medoids and len(top_medoids) > 0:
                print("\nPerforming bimonthly out-of-sample comparison with K-means clustering...")
                bimonthly_sharpe_df = bimonthly_out_of_sample_comparison(
                    market_data, 
                    best_short, 
                    best_long, 
                    top_medoids,
                    big_point_value=big_point_value,
                    slippage=slippage,
                    capital=TRADING_CAPITAL,
                    atr_period=ATR_PERIOD,
                    analysis_type="kmeans"
                )
                
                # Calculate win percentage from bimonthly comparison
                if bimonthly_sharpe_df is not None:
                    total_periods = len(bimonthly_sharpe_df)
                    if total_periods > 0:
                        rounded_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df['Best_sharpe_rounded'])
                        win_percentage = (rounded_wins / total_periods) * 100
                        
                        # Save K-means results to Excel
                        save_to_excel(SYMBOL, win_percentage, top_medoids, best_short, best_long, is_kmeans=True)
            else:
                print("No K-means top medoids found. Cannot run bimonthly comparison.")

            print("\nProceeding with hierarchical cluster analysis...")
            
            # Run the hierarchical cluster analysis to get medoids
            X_filtered_h, medoids_h, top_medoids_h, max_sharpe_point_h, labels_h = hierarchical_cluster_analysis()

            if X_filtered_h is None or medoids_h is None:
                print("Error: Hierarchical cluster analysis failed.")
                continue

            # Re-sort top_medoids to ensure they're in the right order
            if top_medoids_h is not None:
                print("Re-sorting hierarchical top medoids by Sharpe ratio...")
                top_medoids_h = sorted(top_medoids_h, key=lambda x: float(x[2]), reverse=True)
                for idx, medoid in enumerate(top_medoids_h, 1):
                    print(f"Verified Hierarchical Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                        f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

            print("\nPlotting hierarchical strategy performance...")

            # Plot strategy performance with hierarchical results
            market_data_h = plot_strategy_performance(
                best_short, best_long, top_medoids_h, 
                big_point_value=big_point_value,
                slippage=slippage,
                capital=TRADING_CAPITAL,
                atr_period=ATR_PERIOD,
                analysis_type="hierarchical"
            )
            
            # Run the bimonthly out-of-sample comparison for hierarchical
            if top_medoids_h and len(top_medoids_h) > 0:
                print("\nPerforming bimonthly out-of-sample comparison with hierarchical clustering...")
                bimonthly_sharpe_df_h = bimonthly_out_of_sample_comparison(
                    market_data_h, 
                    best_short, 
                    best_long, 
                    top_medoids_h,
                    big_point_value=big_point_value,
                    slippage=slippage,
                    capital=TRADING_CAPITAL,
                    atr_period=ATR_PERIOD,
                    analysis_type="hierarchical"
                )
                
                # Calculate win percentage from bimonthly comparison
                if bimonthly_sharpe_df_h is not None:
                    total_periods = len(bimonthly_sharpe_df_h)
                    if total_periods > 0:
                        rounded_wins = sum(bimonthly_sharpe_df_h['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df_h['Best_sharpe_rounded'])
                        win_percentage = (rounded_wins / total_periods) * 100
                        
                        # Save hierarchical results to Excel
                        save_to_excel(SYMBOL, win_percentage, top_medoids_h, best_short, best_long, is_kmeans=False)
            else:
                print("No hierarchical top medoids found. Cannot run bimonthly comparison.")

            print(f"\nCompleted processing {current_ticker}")

        except Exception as e:
            print(f"\nError processing {current_ticker}: {str(e)}")
            print("Continuing with next underlying...")
            continue

    print("\nAll underlyings processed!")